In [ ]:
pip install jupyter_contrib_nbextensions

In [ ]:
pip show jupyter_contrib_nbextensions

In [ ]:
!jupyter contrib nbextension install

In [ ]:
!jupyter contrib nbextension install --user

In [ ]:
pip install hide_code

In [ ]:
pip show hide_code

In [ ]:
!jupyter nbextension enable --py --sys-prefix hide_code

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import seaborn as sns
import locale

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('float_format', '{:.2f}'.format)
import warnings
warnings.filterwarnings('ignore')

# Definir a localidade para português do Brasil (ou outra localidade que use a vírgula como separador de milhares)
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

'pt_BR.UTF-8'

## Recuperar a base empilhada

In [2]:
file = 'df_empilhada.csv'
df1 = pd.read_csv(file, sep = ';', encoding = 'ISO-8859-1')
df1.shape

(33295732, 56)

In [29]:
df1.shape

(33295732, 41)

# Tratamento de missing e outliers

In [3]:
#ficar somente com as vars que vou usar
keep_vars = [ 
 'NU_ANO',
 'SG_UF_RESIDENCIA',
 'NU_IDADE',
 'TP_SEXO',
 'TP_ESTADO_CIVIL',
 'TP_COR_RACA',
 'TP_ST_CONCLUSAO',
 'TP_ANO_CONCLUIU',
 'TP_ESCOLA',
 'presenca_todas',
 'NU_NOTA_CN',
 'NU_NOTA_CH',
 'NU_NOTA_LC',
 'NU_NOTA_MT',
 'TP_STATUS_REDACAO',
 'NU_NOTA_REDACAO',
 'Q001',
 'Q002',
 'Q003',
 'Q004',
 'Q005',
 'Q006',
 'Q007',
 'Q008',
 'Q009',
 'Q010',
 'Q011',
 'Q012',
 'Q013',
 'Q014',
 'Q015',
 'Q016',
 'Q017',
 'Q018',
 'Q019',
 'Q020',
 'Q021',
 'Q022',
 'Q023',
 'Q024',
 'Q025']

In [4]:
df1 = df1[keep_vars]

## Análise de missing 

In [ ]:
missing_filled = df1[(df1 == -1).any(axis=1)]

In [ ]:
# 694664 = 2% da base tem missing em todas as colunas, considerando apenas as que vou usar
missing_filled.describe()

## Excluindo as linhas com missing da base - aprox 3%

In [ ]:
df1 = df1[~(df1 == -1).any(axis=1)]

In [ ]:
df1.shape

In [ ]:
df1['Q006'].value_counts()

In [ ]:
df1['Q025'].value_counts()

In [ ]:
df1['Q024'].value_counts()

In [ ]:
df1 = df1[~(df1 == '-1').any(axis=1)]

In [ ]:
df1.shape

In [ ]:
#guardar em df1 a base de presentes sem missing
df=df1

In [ ]:
#df.info()

## salvar a base empilhada sem missing

In [ ]:
df.to_csv('df_presentes_sm.csv', sep = ';', index = False)

In [ ]:
file = 'df_presentes_sm.csv'
df = pd.read_csv(file, sep = ';', encoding = 'ISO-8859-1')
df.shape


## 
## 
## 
## 
## 
# Criação de Variáveis e flags 

In [ ]:
# a maior parte já concluiu o ensino medio 59% da amostra e so 27% da amostra esta cursando e vai concluir no ano
vol = df.groupby(['TP_ST_CONCLUSAO']).size().reset_index(name = 'qtde')
vol

In [ ]:
#TP_ST_CONCLUSAO == 2 - estou cursando o EM e concluirei neste ano da prova
df['concluinte_3ano']=np.where(df.TP_ST_CONCLUSAO == 2, 1, 0)

In [ ]:
df2 = pd.crosstab(df['concluinte_3ano'], df['TP_ST_CONCLUSAO'])

In [ ]:
df2

In [ ]:
freqconcluinte_3ano = df['concluinte_3ano'].value_counts()
print("\nFrequência dos valores da coluna 'concluinte_3ano':")
print(freqconcluinte_3ano)

# Marcar os alunos Baixa Renda - Renda Familiar <= 1.5 salários minimos¶

In [ ]:
#Q006 renda familiar

#A	Nenhuma Renda
#B	Até R$ 1.100,00
#C	De R$ 1.100,01 até R$ 1.650,00.
#D	De R$ 1.650,01 até R$ 2.200,00.
#E	De R$ 2.200,01 até R$ 2.750,00.
#F	De R$ 2.750,01 até R$ 3.300,00.
#G	De R$ 3.300,01 até R$ 4.400,00.
#H	De R$ 4.400,01 até R$ 5.500,00.
#I	De R$ 5.500,01 até R$ 6.600,00.
#J	De R$ 6.600,01 até R$ 7.700,00.
#K	De R$ 7.700,01 até R$ 8.800,00.
#L	De R$ 8.800,01 até R$ 9.900,00.
#M	De R$ 9.900,01 até R$ 11.000,00.
#N	De R$ 11.000,01 até R$ 13.200,00.
#O	De R$ 13.200,01 até R$ 16.500,00.
#P	De R$ 16.500,01 até R$ 22.000,00.
#Q	Acima de R$ 22.000,00.

In [ ]:
#df = df.drop(columns=['baixarenda'])

In [ ]:
# Função para categorizar renda menor de 1.5 SM
def categorizar_renda(Q006):
    if Q006 in ('A', 'B', 'C'):
        return 1
    else:
        return 0

In [ ]:
df['baixarenda'] = df['Q006'].apply(categorizar_renda)

In [ ]:
# Função para categorizar renda menor de 1.5 SM - ele entende essa funcao very crazy
#def categorizar_renda(Q006):
#    if Q006 in ('ABC'):
#        return 1
#    else:
#        return 0

In [ ]:
df2= df.groupby(['baixarenda','Q006']).size().reset_index(name = 'qt')

In [ ]:
df2

In [ ]:
df3= df.groupby(['baixarenda','concluinte_3ano']).size().reset_index(name = 'qt')

In [ ]:
df3

## Criar labels para as questões

In [ ]:
# para listar os elementos da variavel (dominio da variavel)
df.Q001.unique()

In [ ]:
df['Q001'].value_counts()

In [ ]:
#Q002 - estudo da mae
#A	Nunca estudou.
#B	Não completou a 4ª série/5º ano do Ensino Fundamental.
#C	Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental.
#D	Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio.
#E	Completou o Ensino Médio, mas não completou a Faculdade.
#F	Completou a Faculdade, mas não completou a Pós-graduação.
#G	Completou a Pós-graduação.
#H	Não sei.

In [ ]:
#Q001 - estudo do pai
#A	Nunca estudou.
#B	Não completou a 4ª série/5º ano do Ensino Fundamental.
#C	Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental.
#D	Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio.
#E	Completou o Ensino Médio, mas não completou a Faculdade.
#F	Completou a Faculdade, mas não completou a Pós-graduação.
#G	Completou a Pós-graduação.
#H	Não sei.

In [ ]:
df.Q002.unique()

In [ ]:
map_Q001_2 = {
'A' : 1,
'B' : 2,
'C' : 3,
'D' : 4,
'E' : 5,
'F' : 6,
'G' : 7,
'H' : 0,
-1  : -1    
}

In [ ]:
#criar a variavel cat_Q001 para substituir A-H por numeros
df['cat_Q001'] = df['Q001'].map(map_Q001_2)
df['cat_Q002'] = df['Q002'].map(map_Q001_2)

In [ ]:
#Q003 profissao do pai
#A	Grupo 1: Lavrador, agricultor sem empregados, bóia fria, criador de animais (gado, porcos, galinhas, ovelhas, cavalos etc.), apicultor, pescador, lenhador, seringueiro, extrativista.
#B	Grupo 2: Diarista, empregado doméstico, cuidador de idosos, babá, cozinheiro (em casas particulares), motorista particular, jardineiro, faxineiro de empresas e prédios, vigilante, porteiro, carteiro, office-boy, vendedor, caixa, atendente de loja, auxiliar administrativo, recepcionista, servente de pedreiro, repositor de mercadoria.
#C	Grupo 3: Padeiro, cozinheiro industrial ou em restaurantes, sapateiro, costureiro, joalheiro, torneiro mecânico, operador de máquinas, soldador, operário de fábrica, trabalhador da mineração, pedreiro, pintor, eletricista, encanador, motorista, caminhoneiro, taxista.
#D	Grupo 4: Professor (de ensino fundamental ou médio, idioma, música, artes etc.), técnico (de enfermagem, contabilidade, eletrônica etc.), policial, militar de baixa patente (soldado, cabo, sargento), corretor de imóveis, supervisor, gerente, mestre de obras, pastor, microempresário (proprietário de empresa com menos de 10 empregados), pequeno comerciante, pequeno proprietário de terras, trabalhador autônomo ou por conta própria.
#E	Grupo 5: Médico, engenheiro, dentista, psicólogo, economista, advogado, juiz, promotor, defensor, delegado, tenente, capitão, coronel, professor universitário, diretor em empresas públicas ou privadas, político, proprietário de empresas com mais de 10 empregados.
#F	Não sei.

In [ ]:
df.Q003.unique()

In [ ]:
#Q004 profissao da mae
#A	Grupo 1: Lavradora, agricultora sem empregados, bóia fria, criadora de animais (gado, porcos, galinhas, ovelhas, cavalos etc.), apicultora, pescadora, lenhadora, seringueira, extrativista.
#B	Grupo 2: Diarista, empregada doméstica, cuidadora de idosos, babá, cozinheira (em casas particulares), motorista particular, jardineira, faxineira de empresas e prédios, vigilante, porteira, carteira, office-boy, vendedora, caixa, atendente de loja, auxiliar administrativa, recepcionista, servente de pedreiro, repositora de mercadoria.
#C	Grupo 3: Padeira, cozinheira industrial ou em restaurantes, sapateira, costureira, joalheira, torneira mecânica, operadora de máquinas, soldadora, operária de fábrica, trabalhadora da mineração, pedreira, pintora, eletricista, encanadora, motorista, caminhoneira, taxista.
#D	Grupo 4: Professora (de ensino fundamental ou médio, idioma, música, artes etc.), técnica (de enfermagem, contabilidade, eletrônica etc.), policial, militar de baixa patente (soldado, cabo, sargento), corretora de imóveis, supervisora, gerente, mestre de obras, pastora, microempresária (proprietária de empresa com menos de 10 empregados), pequena comerciante, pequena proprietária de terras, trabalhadora autônoma ou por conta própria.
#E	Grupo 5: Médica, engenheira, dentista, psicóloga, economista, advogada, juíza, promotora, defensora, delegada, tenente, capitã, coronel, professora universitária, diretora em empresas públicas ou privadas, política, proprietária de empresas com mais de 10 empregados.
#F	Não sei.

In [ ]:
df.Q004.unique()

In [ ]:
map_Q003_4 = {
'A' : 1,
'B' : 2,
'C' : 3,
'D' : 4,
'E' : 5,
'F' : 0,
-1  : -1
}

In [ ]:
df['cat_Q003'] = df['Q003'].map(map_Q003_4)
df['cat_Q004'] = df['Q004'].map(map_Q003_4)

In [ ]:
#Q006 renda familiar

#A	Nenhuma Renda
#B	Até R$ 1.100,00
#C	De R$ 1.100,01 até R$ 1.650,00.
#D	De R$ 1.650,01 até R$ 2.200,00.
#E	De R$ 2.200,01 até R$ 2.750,00.
#F	De R$ 2.750,01 até R$ 3.300,00.
#G	De R$ 3.300,01 até R$ 4.400,00.
#H	De R$ 4.400,01 até R$ 5.500,00.
#I	De R$ 5.500,01 até R$ 6.600,00.
#J	De R$ 6.600,01 até R$ 7.700,00.
#K	De R$ 7.700,01 até R$ 8.800,00.
#L	De R$ 8.800,01 até R$ 9.900,00.
#M	De R$ 9.900,01 até R$ 11.000,00.
#N	De R$ 11.000,01 até R$ 13.200,00.
#O	De R$ 13.200,01 até R$ 16.500,00.
#P	De R$ 16.500,01 até R$ 22.000,00.
#Q	Acima de R$ 22.000,00.

In [ ]:
df.Q006.unique()

In [ ]:
map_Q006 = {
-1  : -1,
'A' : 0,
'B' : 1,
'C' : 1.5,
'D' : 2,
'E' : 2.5,
'F' : 3,
'G' : 4,
'H' : 5,    
'I' : 6,
'J' : 7,
'K' : 8,
'L' : 9,    
'M' : 10,    
'N' : 12,    
'O' : 15,
'P' : 20,
'Q' : 30 
}

In [ ]:
df['cat_Q006'] = df['Q006'].map(map_Q006)

In [ ]:
#Q007 - se ha empregada domestica em casa
#A	Não.
#B	Sim, um ou dois dias por semana.
#C	Sim, três ou quatro dias por semana.
#D	Sim, pelo menos cinco dias por semana.

In [ ]:
df.Q007.unique()

In [ ]:
map_Q007 = {
-1  : -1,
'A' : 0,
'B' : 1,
'C' : 2,
'D' : 3  
}

In [ ]:
df['cat_Q007'] = df['Q007'].map(map_Q007)

In [ ]:
#Q008 - qtde de banheiros em casa
#A	Não.
#B	Sim, um.
#C	Sim, dois.
#D	Sim, três.
#E	Sim, quatro ou mais.

In [ ]:
#Q008	Na sua residência tem banheiro?
#Q009	Na sua residência tem quartos para dormir?
#Q010	Na sua residência tem carro?
#Q011	Na sua residência tem motocicleta?
#Q012	Na sua residência tem geladeira?
#Q013	Na sua residência tem freezer (independente ou segunda porta da geladeira)?
#Q014	Na sua residência tem máquina de lavar roupa? (o tanquinho NÃO deve ser considerado)
#Q015	Na sua residência tem máquina de secar roupa (independente ou em conjunto com a máquina de lavar roupa)?
#Q016	Na sua residência tem forno micro-ondas?
#Q017	Na sua residência tem máquina de lavar louça?

In [ ]:
df.Q008.unique()

In [ ]:
map_Q00_8a17_19_22_24 = {
-1  : -1,
'A' : 0,
'B' : 1,
'C' : 2,
'D' : 3,
'E' : 4  
}

In [ ]:
df['cat_Q008'] = df['Q008'].map(map_Q00_8a17_19_22_24)
df['cat_Q009'] = df['Q009'].map(map_Q00_8a17_19_22_24)
df['cat_Q010'] = df['Q010'].map(map_Q00_8a17_19_22_24)
df['cat_Q011'] = df['Q011'].map(map_Q00_8a17_19_22_24)
df['cat_Q012'] = df['Q012'].map(map_Q00_8a17_19_22_24)
df['cat_Q013'] = df['Q013'].map(map_Q00_8a17_19_22_24)
df['cat_Q014'] = df['Q014'].map(map_Q00_8a17_19_22_24)
df['cat_Q015'] = df['Q015'].map(map_Q00_8a17_19_22_24)
df['cat_Q016'] = df['Q016'].map(map_Q00_8a17_19_22_24)
df['cat_Q017'] = df['Q017'].map(map_Q00_8a17_19_22_24)
df['cat_Q019'] = df['Q019'].map(map_Q00_8a17_19_22_24)
df['cat_Q022'] = df['Q022'].map(map_Q00_8a17_19_22_24)
df['cat_Q024'] = df['Q024'].map(map_Q00_8a17_19_22_24)

In [ ]:
#Q018	Na sua residência tem aspirador de pó?
#Q020	Na sua residência tem aparelho de DVD?
#Q021	Na sua residência tem TV por assinatura?
#Q023	Na sua residência tem telefone fixo?
#Q025	Na sua residência tem acesso à Internet?

In [ ]:
df.Q018.unique()

In [ ]:
map_Q01820212325 = {
-1  :-1,
'A' : 0,
'B' : 1  
}

In [ ]:
df['cat_Q018'] = df['Q018'].map(map_Q01820212325)
df['cat_Q020'] = df['Q020'].map(map_Q01820212325)
df['cat_Q021'] = df['Q021'].map(map_Q01820212325)
df['cat_Q023'] = df['Q023'].map(map_Q01820212325)
df['cat_Q025'] = df['Q025'].map(map_Q01820212325)

## Criar variável Feminimo

In [ ]:
df['FEMININO']=np.where(df.TP_SEXO == "F", 1, 0)

In [ ]:
pd.crosstab(df.TP_SEXO, df.FEMININO)

## Criar variáveis categóricas para Região

In [ ]:
df['SG_UF_RESIDENCIA'].value_counts()

In [ ]:
map_REG = {
'MG' :'Sudeste',
'ES' :'Sudeste',
'RJ' :'Sudeste',
'SP' :'Sudeste',
'RR' :'Norte',
'AP' :'Norte',
'AM' :'Norte',
'PA' :'Norte',
'AC' :'Norte',
'RO' :'Norte',
'TO' :'Norte',
'MA' :'Nordeste',
'PI' :'Nordeste',
'CE' :'Nordeste',
'RN' :'Nordeste',
'PB' :'Nordeste',
'PE' :'Nordeste',
'AL' :'Nordeste',
'SE' :'Nordeste',
'BA' :'Nordeste',
'MT' :'Centro-oeste',
'DF' :'Centro-oeste',
'GO' :'Centro-oeste',
'MS' :'Centro-oeste',
'PR' :'Sul',
'SC' :'Sul',
'RS' :'Sul'
}

In [ ]:
df['REG_UF'] = df['SG_UF_RESIDENCIA'].map(map_REG)

In [ ]:
map_Norte = {
'Norte' : 1,
'Centro-oeste' : 0,
'Sudeste' : 0,
'Nordeste' : 0,
'Sul' : 0,  
}

map_Nordeste = {
'Norte' : 0,
'Centro-oeste' : 0,
'Sudeste' : 0,
'Nordeste' : 1,
'Sul' : 0,  
}

map_Sul = {
'Norte' : 0,
'Centro-oeste' : 0,
'Sudeste' : 0,
'Nordeste' : 0,
'Sul' : 1,  
}

map_CentroOeste = {
'Norte' : 0,
'Centro-oeste' : 1,
'Sudeste' : 0,
'Nordeste' : 0,
'Sul' : 0,  
}

map_Sudeste = {
'Norte' : 0,
'Centro-oeste' : 0,
'Sudeste' : 1,
'Nordeste' : 0,
'Sul' : 0,  
}

In [ ]:
df['cat_Norte'] = df['REG_UF'].map(map_Norte)
df['cat_Nordeste'] = df['REG_UF'].map(map_Nordeste)
df['cat_Sul'] = df['REG_UF'].map(map_Sul)
df['cat_CentroOeste'] = df['REG_UF'].map(map_CentroOeste)
df['cat_Sudeste'] = df['REG_UF'].map(map_Sudeste)

## Criar variavel tem_TIC quando o aluno tem acesso a internet e tem computador em casa

In [ ]:
df.Q024.unique()

In [ ]:
df.cat_Q024.unique()

In [ ]:
df.shape

In [ ]:
df['tem_TIC'] = np.where((df.cat_Q024 != 0) &
                         (df.cat_Q025 == 1) , 'PC_NET',
                         np.where((df.cat_Q024 == 0) &
                                  (df.cat_Q025 == 0), 'NAO_TEM',
                                   np.where((df.cat_Q024 != 0) &
                                            (df.cat_Q025 == 0), 'TEM_PC',
                                            np.where((df.cat_Q024 == 0) &
                                                      (df.cat_Q025 == 1), 'TEM_NET','MISSING'))))

In [ ]:
df.tem_TIC.unique()

In [ ]:
df['tem_TICv2'] = np.where((df.cat_Q025 == 0), 'NI',
                          np.where((df.cat_Q025 == 1) & 
                                    (df.cat_Q024 != 0) &
                                    (df.cat_Q022 != 0), 'I_Comp_Cel',
                                    np.where((df.cat_Q025 == 1) &
                                             (df.cat_Q024 != 0) &
                                             (df.cat_Q022 == 0), 'I_Comp_nCel',
                                             np.where((df.cat_Q025 == 1) &
                                                      (df.cat_Q024 == 0) &
                                                      (df.cat_Q022 != 0), 'I_nComp_Cel',
                                                      np.where((df.cat_Q025 == 1),'SoI' , 'MISSING')))))

In [ ]:
df.tem_TICv2.unique()

In [ ]:
df['tem_TICv3'] = np.where((df.cat_Q025 == 0), 'NI',#nao tem acesso a internet
                          np.where((df.cat_Q025 == 1) & 
                                    (df.cat_Q024 != 0) &
                                    (df.cat_Q024 != -1), 'IeC',#tem acesso a internet e computador
                                    np.where((df.cat_Q025 == 1) &
                                             (df.cat_Q024 == 0), 'IenC', 'MISSING')))#tem acesso a internet mas não tem computador

In [ ]:
df.tem_TICv3.unique()

In [ ]:
map_TIC = {
'MISSING' : 0,
'NAO_TEM' : 1,
'TEM_NET' : 2, 
'TEM_PC'  : 3, 
'PC_NET'  : 4
}

In [ ]:
df['TIC'] = df['tem_TIC'].map(map_TIC)



df['I'] =  np.where(df.TIC == 1, 0,
                       np.where(df.TIC == 2, 1,
                                np.where(df.TIC == 3, 0,
                                        np.where(df.TIC == 4, 1, 0))))
                                         
                                         
df['C'] =  np.where(df.TIC == 1, 0,
                       np.where(df.TIC == 2, 0,
                                np.where(df.TIC == 3, 1,
                                        np.where(df.TIC == 4, 1, 0))))

In [ ]:
df['sóI'] = df['I'] * (1 - df['C'])
df['sóC'] = df['C'] * (1 - df['I'])
df['IeC'] = df['I'] * df['C']

# Criando a marcação do publico alvo na base final 

## público alvo = alunos de baixa renda e no último ano do ensino médio

In [ ]:
# Adicionando uma nova coluna "marcacao" (alunos cursando 3 ano e de baixa renda - ~5.1 MM)
df['marcacao'] = np.where((df['concluinte_3ano'] == 1) & (df['baixarenda'] == 1), 1, 0)

In [ ]:
df['marcacao'].value_counts()

In [ ]:
df.shape

In [ ]:
list(df)

In [ ]:
to_keep_finals_features = [
 'NU_ANO',
 #'SG_UF_RESIDENCIA',
 'NU_IDADE',
# 'TP_SEXO',
 'TP_ESTADO_CIVIL',
 'TP_COR_RACA',
# 'TP_NACIONALIDADE',
 'TP_ST_CONCLUSAO',
 'TP_ANO_CONCLUIU',
 'TP_ESCOLA', #tipo da escola#
 #'TP_PRESENCA_MT',
 #'TP_PRESENCA_CN',
 #'TP_PRESENCA_CH',
 #'TP_PRESENCA_LC',
 #'presenca_todas',
 'NU_NOTA_CN',
 'NU_NOTA_CH',
 'NU_NOTA_LC',
 'NU_NOTA_MT',
 #'TP_STATUS_REDACAO',
 'NU_NOTA_REDACAO',
# 'CO_MUNICIPIO_PROVA',
# 'NO_MUNICIPIO_PROVA',
# 'CO_UF_PROVA',
# 'SG_UF_PROVA',
 'Q001',
 'Q002',
 #'Q003',
 #'Q004',
 'Q005', #tamanho da familia#
 #'Q006',
 #'Q007',
 #'Q008',
 #'Q009',
 #'Q010',
 #'Q011',
 #'Q012',
 #'Q013',
 #'Q014',
 #'Q015',
 #'Q016',
 #'Q017',
 #'Q018',
 #'Q019',
 #'Q020',
 'Q021',
 'Q022',
 #'Q023',
 'Q024',
 'Q025',
 #'NU_IDADE1',
 #'NU_NOTA_MT_acima1000',
 #'NU_NOTA_MT_1',
 'concluinte_3ano',
 'baixarenda',
 'marcacao',
 'cat_Q001',#escolar pai#
 'cat_Q002', #escolar mae#
 'cat_Q003',
 'cat_Q004',
 'cat_Q006',
 'cat_Q007',
 'cat_Q008',
 'cat_Q009',
 'cat_Q010',
 'cat_Q011',
 'cat_Q012',
 'cat_Q013',
 'cat_Q014',
 'cat_Q015',
 'cat_Q016',
 'cat_Q017',
 'cat_Q019',
 'cat_Q022',
 'cat_Q024',
 'cat_Q018',
 'cat_Q020',
 'cat_Q021',
 'cat_Q023',
 'cat_Q025',
 'FEMININO',
 'REG_UF',
 'cat_Norte',
 'cat_Nordeste',
 'cat_Sul',
 'cat_CentroOeste',
 'cat_Sudeste',
 'tem_TIC',
 'tem_TICv2',
 'tem_TICv3',
 'TIC',
 'I',
 'C',
 'sóI',
 'sóC',
 'IeC'
]

In [ ]:
df_final = df[to_keep_finals_features]

In [ ]:
df_final.shape

In [ ]:
print(df_final[['marcacao']].head())

In [ ]:
#df_validacao = df_final.groupby(['marcacao']).size().reset_index(name='counts')

In [ ]:
#df_validacao

In [ ]:
#df_validacao = df_final.groupby(['NU_ANO','marcacao', 'baixarenda', 'concluinte_3ano']).size().reset_index(name='qtd')

In [ ]:
#df_validacao

In [ ]:
#df_validacao = df_final.groupby(['NU_ANO', 'concluinte_3ano']).size().reset_index(name='counts')

In [ ]:
#df_validacao

In [ ]:
#df_validacao = df_final.groupby(['NU_ANO', 'baixarenda']).size().reset_index(name='counts')

In [ ]:
#df_validacao

In [ ]:
#df_final.info()

# Salvando a base final dos alunos presentes com marcação do público

In [ ]:
# save final data base marcada com o publico alvo (baixa renda e ult ano em)
df_final.to_csv('df_presentes_sm_marcada.csv', sep = ';', index = False)

In [12]:
inscritos_ano = df1.groupby(['NU_ANO']).size().reset_index(name='counts')

In [13]:
inscritos_ano

,NU_ANO,counts
0,2015,5604905
1,2016,5818446
2,2017,4426755
3,2018,3893743
4,2019,3702008
5,2020,2588681
6,2021,2238107
7,2022,2344823
8,2023,2678264


In [37]:
df_validacao = df1[df1.TP_ST_CONCLUSAO == 2]

In [38]:
df_validacao.TP_ST_CONCLUSAO.describe()

count   10484719.00
mean           2.00
std            0.00
min            2.00
25%            2.00
50%            2.00
75%            2.00
max            2.00
Name: TP_ST_CONCLUSAO, dtype: float64

# inscritos x 3 ano  

In [ ]:
df_inscritos = df1.groupby(['NU_ANO', 'TP_ST_CONCLUSAO']).size().reset_index(name='counts')

## presentes x 3 ano

In [9]:
file = 'df_presentes_sm_marcada.csv'
df2 = pd.read_csv(file, sep = ';', encoding = 'ISO-8859-1')
df2.shape

(32601016, 56)

In [24]:
df_validacao = df2.groupby(['NU_ANO', 'REG_UF']).size().reset_index(name='counts')

In [25]:
df_validacao

,NU_ANO,REG_UF,counts
0,2015,Centro-oeste,468658
1,2015,Nordeste,1788358
2,2015,Norte,551796
3,2015,Sudeste,1967749
4,2015,Sul,634471
5,2016,Centro-oeste,474799
6,2016,Nordeste,1904434
7,2016,Norte,609762
8,2016,Sudeste,1981374
9,2016,Sul,643618


In [26]:
df_validacao = df3.groupby(['NU_ANO', 'REG_UF']).size().reset_index(name='counts')

In [27]:
df_validacao

,NU_ANO,REG_UF,counts
0,2015,Centro-oeste,122874
1,2015,Nordeste,384666
2,2015,Norte,122814
3,2015,Sudeste,582661
4,2015,Sul,187586
5,2016,Centro-oeste,128930
6,2016,Nordeste,415398
7,2016,Norte,141479
8,2016,Sudeste,598163
9,2016,Sul,206540


In [41]:
map_REG = {
'MG' :'Sudeste',
'ES' :'Sudeste',
'RJ' :'Sudeste',
'SP' :'Sudeste',
'RR' :'Norte',
'AP' :'Norte',
'AM' :'Norte',
'PA' :'Norte',
'AC' :'Norte',
'RO' :'Norte',
'TO' :'Norte',
'MA' :'Nordeste',
'PI' :'Nordeste',
'CE' :'Nordeste',
'RN' :'Nordeste',
'PB' :'Nordeste',
'PE' :'Nordeste',
'AL' :'Nordeste',
'SE' :'Nordeste',
'BA' :'Nordeste',
'MT' :'Centro-oeste',
'DF' :'Centro-oeste',
'GO' :'Centro-oeste',
'MS' :'Centro-oeste',
'PR' :'Sul',
'SC' :'Sul',
'RS' :'Sul'
}

In [42]:
df1['REG_UF'] = df1['SG_UF_RESIDENCIA'].map(map_REG)

In [45]:
df3 = df1[df1['TP_ST_CONCLUSAO'] == 2]

In [46]:
df_validacao = df3.groupby(['NU_ANO', 'REG_UF']).size().reset_index(name='counts')

In [47]:
df_validacao

,NU_ANO,REG_UF,counts
0,2015,Centro-oeste,125265
1,2015,Nordeste,391194
2,2015,Norte,125695
3,2015,Sudeste,590070
4,2015,Sul,191030
5,2016,Centro-oeste,132154
6,2016,Nordeste,423693
7,2016,Norte,145055
8,2016,Sudeste,608000
9,2016,Sul,211144
